In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm
import torch.nn.functional as F
import copy

2025-05-28 20:42:12.509047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748464932.731570      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748464932.800796      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_df = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_train.csv", encoding='iso-8859-1')
test_df = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_test.csv", encoding='iso-8859-1')

In [12]:
train_df.shape

(957, 3)

In [13]:
test_df.shape

(125, 3)

In [10]:
train_df['Label'].value_counts()

Label
Non-Spam    835
Spam        122
Name: count, dtype: int64

In [18]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
train_df['Spam_Label']= label_encoder.fit_transform(train_df['Label'])
test_df['Spam_Label']= label_encoder.fit_transform(test_df['Label'])
train_df.head

<bound method NDFrame.head of      S. No.                                       Message_body     Label  \
0         1                         Rofl. Its true to its name  Non-Spam   
1         2  The guy did some bitching but I acted like i'd...  Non-Spam   
2         3  Pity, * was in mood for that. So...any other s...  Non-Spam   
3         4               Will ü b going to esplanade fr home?  Non-Spam   
4         5  This is the 2nd time we have tried 2 contact u...      Spam   
..      ...                                                ...       ...   
952     953  hows my favourite person today? r u workin har...  Non-Spam   
953     954                      How much you got for cleaning  Non-Spam   
954     955  Sorry da. I gone mad so many pending works wha...  Non-Spam   
955     956                                 Wat time ü finish?  Non-Spam   
956     957                    Just glad to be talking to you.  Non-Spam   

     Spam_Label  
0             0  
1             0  
2  

In [19]:
test_df.head

<bound method NDFrame.head of      S. No.                                       Message_body     Label  \
0         1  UpgrdCentre Orange customer, you may now claim...      Spam   
1         2  Loan for any purpose £500 - £75,000. Homeowner...      Spam   
2         3  Congrats! Nokia 3650 video camera phone is you...      Spam   
3         4  URGENT! Your Mobile number has been awarded wi...      Spam   
4         5  Someone has contacted our dating service and e...      Spam   
..      ...                                                ...       ...   
120     121  7 wonders in My WORLD 7th You 6th Ur style 5th...  Non-Spam   
121     122  Try to do something dear. You read something f...  Non-Spam   
122     123  Sun ah... Thk mayb can if dun have anythin on....  Non-Spam   
123     124  SYMPTOMS when U are in love: "1.U like listeni...  Non-Spam   
124     125  Great. Have a safe trip. Dont panic surrender ...  Non-Spam   

     Spam_Label  
0             1  
1             1  
2  

In [ ]:
df = pd.concat(train_df, test_df)